In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder \
    .appName("RetailDemandBaselineModel") \
    .getOrCreate()


26/01/16 14:54:32 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 10.0.0.22 instead (on interface en0)
26/01/16 14:54:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/16 14:54:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/16 14:54:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/16 14:54:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df = spark.read.parquet("../data/processed/daily_features")
df.printSchema()
df.show(5)


root
 |-- date: date (nullable = true)
 |-- daily_quantity: long (nullable = true)
 |-- lag_1: long (nullable = true)
 |-- lag_7: long (nullable = true)
 |-- rolling_7: double (nullable = true)
 |-- rolling_14: double (nullable = true)

+----------+--------------+-----+-----+------------------+------------------+
|      date|daily_quantity|lag_1|lag_7|         rolling_7|        rolling_14|
+----------+--------------+-----+-----+------------------+------------------+
|2010-12-01|         26919| NULL| NULL|              NULL|              NULL|
|2010-12-02|         31329|26919| NULL|           26919.0|           26919.0|
|2010-12-03|         16199|31329| NULL|           29124.0|           29124.0|
|2010-12-05|         16450|16199| NULL|24815.666666666668|24815.666666666668|
|2010-12-06|         21795|16450| NULL|          22724.25|          22724.25|
+----------+--------------+-----+-----+------------------+------------------+
only showing top 5 rows



In [6]:
df_model = df.dropna()
df_model.count()


298

In [7]:
from pyspark.sql.functions import col

df_clean = df.select([
    col(c).alias(c.lower().replace(" ", "_")) for c in df.columns
])
